In [1]:
from torchvision.datasets import Cityscapes
from torchvision import transforms
from torch.utils.data import DataLoader, Subset

image_size = (64, 128)
root_dir = "datasets/cityscapes"

input_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
])

target_transform = transforms.Compose([
    transforms.Resize(
        image_size,
        interpolation=transforms.InterpolationMode.NEAREST
    )
])


full_train_dataset = Cityscapes(
    root=root_dir,
    split='train',
    mode='fine',
    target_type='semantic',
    transform=input_transform,
    target_transform=target_transform,
)

full_val_dataset = Cityscapes(
    root=root_dir,
    split='val',
    mode='fine',
    target_type='semantic',
    transform=input_transform,
    target_transform=target_transform,
)

from utils.Cityscapes.CityscapesWrapper import CityscapesWrapper

train_samples = 800
val_samples = 200

train_subset = CityscapesWrapper(
    Subset(
        full_train_dataset,
        range(train_samples)
    ),
    target_transform=target_transform
)
val_subset = CityscapesWrapper(
    Subset(
        full_val_dataset,
        range(val_samples)
    ),
    target_transform=target_transform
)

train_loader = DataLoader(
    train_subset,
    batch_size=4,
    shuffle=True,
    num_workers=0
)
val_loader = DataLoader(
    val_subset,
    batch_size=4,
    shuffle=False,
    num_workers=0
)

In [ ]:
import torch
import torchattacks
import matplotlib.pyplot as plt
import numpy as np
from utils.Cityscapes.UNetTorchCityscapes import UNet

device = "mps"

single_unet = UNet(in_channels=3, out_channels=34)
single_unet.load_state_dict(torch.load("./weights/cityscapes_unet.pth"))
single_unet.to(device)
single_unet.eval()

ensemble_models = []
for i in range(3):
    ensemble_model = UNet(in_channels=3, out_channels=34)
    ensemble_model.load_state_dict(torch.load(f"./weights/cityscapes_ensemble_unet_{i+1}.pth"))
    ensemble_model.to(device)
    ensemble_model.eval()
    ensemble_models.append(ensemble_model)

def apply_colormap(mask):
    colormap = np.array([
        [0, 0, 0],         # class 0: black
        [0, 255, 0],       # class 1: green
        [0, 0, 255],       # class 2: blue
        [255, 0, 0],       # class 3: red
        [255, 255, 0],     # etc.
    ])
    mask_rgb = colormap[mask % len(colormap)]
    return mask_rgb.astype(np.uint8)

sample_image, sample_target = next(iter(val_loader))
sample_image = sample_image.to(device)
sample_target = sample_target.to(device).long().squeeze(1)

with torch.no_grad():
    single_unet_output = single_unet(sample_image)
    single_unet_pred = torch.argmax(single_unet_output, dim=1)

ensemble_preds = [torch.softmax(m(sample_image), dim=1) for m in ensemble_models]
ensemble_avg_pred = torch.stack(ensemble_preds).mean(dim=0)
ensemble_pred = torch.argmax(ensemble_avg_pred, dim=1)

fgsm = torchattacks.FGSM(single_unet, eps=0.02)
pgd = torchattacks.PGD(single_unet, eps=0.02, alpha=0.01, steps=40)


sample_fgsm = fgsm(sample_image, sample_target)
sample_pgd = pgd(sample_image, sample_target)


with torch.no_grad():
    fgsm_single_output = single_unet(sample_fgsm)
    fgsm_single_pred = torch.argmax(fgsm_single_output, dim=1)

    pgd_single_output = single_unet(sample_pgd)
    pgd_single_pred = torch.argmax(pgd_single_output, dim=1)


fgsm_ensemble_preds = [torch.softmax(m(sample_fgsm), dim=1) for m in ensemble_models]
fgsm_ensemble_avg_pred = torch.stack(fgsm_ensemble_preds).mean(dim=0)
fgsm_ensemble_pred = torch.argmax(fgsm_ensemble_avg_pred, dim=1)

pgd_ensemble_preds = [torch.softmax(m(sample_pgd), dim=1) for m in ensemble_models]
pgd_ensemble_avg_pred = torch.stack(pgd_ensemble_preds).mean(dim=0)
pgd_ensemble_pred = torch.argmax(pgd_ensemble_avg_pred, dim=1)

img = sample_image[0].cpu().permute(1, 2, 0).numpy()
gt_mask = sample_target[0].cpu().squeeze().numpy()
single_unet_mask = single_unet_pred[0].cpu().numpy()
ensemble_mask = ensemble_pred[0].cpu().numpy()
fgsm_single_mask = fgsm_single_pred[0].cpu().numpy()
pgd_single_mask = pgd_single_pred[0].cpu().numpy()
fgsm_ensemble_mask = fgsm_ensemble_pred[0].cpu().numpy()
pgd_ensemble_mask = pgd_ensemble_pred[0].cpu().numpy()

fig, axs = plt.subplots(2, 4, figsize=(18, 9))

# Original Image
axs[0, 0].imshow(img)
axs[0, 0].set_title("Input Image")
axs[0, 0].axis('off')

# Ground Truth Mask
axs[0, 1].imshow(apply_colormap(gt_mask))
axs[0, 1].set_title("Ground Truth Mask")
axs[0, 1].axis('off')

# Mask with Single UNet
axs[0, 2].imshow(apply_colormap(single_unet_mask))
axs[0, 2].set_title("Single UNet Prediction")
axs[0, 2].axis('off')

# Mask with Ensemble UNet
axs[0, 3].imshow(apply_colormap(ensemble_mask))
axs[0, 3].set_title("Ensemble UNet Prediction")
axs[0, 3].axis('off')

# Mask with FGSM Attack and Single UNet
axs[1, 0].imshow(apply_colormap(fgsm_single_mask))
axs[1, 0].set_title("FGSM Attack (Single UNet)")
axs[1, 0].axis('off')

# Mask with PGD Attack and Single UNet
axs[1, 1].imshow(apply_colormap(pgd_single_mask))
axs[1, 1].set_title("PGD Attack (Single UNet)")
axs[1, 1].axis('off')

# Mask with FGSM Attack and Ensemble UNet
axs[1, 2].imshow(apply_colormap(fgsm_ensemble_mask))
axs[1, 2].set_title("FGSM Attack (Ensemble UNet)")
axs[1, 2].axis('off')

# Mask with PGD Attack and Ensemble UNet
axs[1, 3].imshow(apply_colormap(pgd_ensemble_mask))
axs[1, 3].set_title("PGD Attack (Ensemble UNet)")
axs[1, 3].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torchattacks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils.Cityscapes.UNetTorchCityscapes import UNet
from sklearn.metrics import jaccard_score, f1_score, accuracy_score
from torchvision.utils import make_grid

device = "mps"

single_unet = UNet(in_channels=3, out_channels=34)
single_unet.load_state_dict(torch.load("./weights/cityscapes_unet.pth"))
single_unet.to(device)
single_unet.eval()


ensemble_models = []
for i in range(3):
    model = UNet(in_channels=3, out_channels=34)
    model.load_state_dict(torch.load(f"./weights/cityscapes_ensemble_unet_{i+1}.pth"))
    model.to(device)
    model.eval()
    ensemble_models.append(model)


fgsm = torchattacks.FGSM(single_unet, eps=0.02)
pgd = torchattacks.PGD(single_unet, eps=0.02, alpha=0.01, steps=40)


def apply_colormap(mask):
    colormap = np.array([
        [0, 0, 0], [0, 255, 0], [0, 0, 255],
        [255, 0, 0], [255, 255, 0], [255, 0, 255],
        [0, 255, 255], [128, 0, 0], [0, 128, 0],
        [0, 0, 128], [128, 128, 0], [128, 0, 128],
        [0, 128, 128], [64, 0, 0], [0, 64, 0],
        [0, 0, 64], [64, 64, 0], [64, 0, 64],
        [0, 64, 64], [192, 192, 192], [128, 128, 128],
        [64, 128, 128], [128, 64, 128], [128, 128, 64],
        [192, 0, 0], [0, 192, 0], [0, 0, 192],
        [192, 192, 0], [192, 0, 192], [0, 192, 192],
        [64, 64, 64], [32, 32, 32], [160, 160, 160],
        [224, 224, 224]
    ])
    return colormap[mask % len(colormap)].astype(np.uint8)


def compute_metrics(pred, gt, num_classes=34):
    pred_flat = pred.flatten()
    gt_flat = gt.flatten()

    iou = jaccard_score(gt_flat, pred_flat, average='macro', labels=range(num_classes), zero_division=0)
    dice = f1_score(gt_flat, pred_flat, average='macro', labels=range(num_classes), zero_division=0)
    pixel_acc = accuracy_score(gt_flat, pred_flat)
    return dice, iou, pixel_acc


results = []

val_iter = iter(val_loader)
for i in range(20):
    sample_image, sample_target = next(val_iter)
    sample_image = sample_image.to(device)
    sample_target = sample_target.to(device).long().squeeze(1)

    with torch.no_grad():
        single_out = single_unet(sample_image)
        single_pred = torch.argmax(single_out, dim=1)

        ensemble_probs = [torch.softmax(m(sample_image), dim=1) for m in ensemble_models]
        ensemble_avg = torch.stack(ensemble_probs).mean(0)
        ensemble_pred = torch.argmax(ensemble_avg, dim=1)

    sample_fgsm = fgsm(sample_image, sample_target)
    sample_pgd = pgd(sample_image, sample_target)

    with torch.no_grad():
        fgsm_single_pred = torch.argmax(single_unet(sample_fgsm), dim=1)
        pgd_single_pred = torch.argmax(single_unet(sample_pgd), dim=1)

        fgsm_ensemble_avg = torch.stack([torch.softmax(m(sample_fgsm), dim=1) for m in ensemble_models]).mean(0)
        pgd_ensemble_avg = torch.stack([torch.softmax(m(sample_pgd), dim=1) for m in ensemble_models]).mean(0)

        fgsm_ensemble_pred = torch.argmax(fgsm_ensemble_avg, dim=1)
        pgd_ensemble_pred = torch.argmax(pgd_ensemble_avg, dim=1)

    modes = {
        "Single": single_pred,
        "Ensemble": ensemble_pred,
        "FGSM_Single": fgsm_single_pred,
        "PGD_Single": pgd_single_pred,
        "FGSM_Ensemble": fgsm_ensemble_pred,
        "PGD_Ensemble": pgd_ensemble_pred
    }

    for mode_name, pred_mask in modes.items():
        dice, iou, acc = compute_metrics(pred_mask[0].cpu().numpy(), sample_target[0].cpu().numpy())
        results.append({
            "Sample": i,
            "Mode": mode_name,
            "Dice": dice,
            "IoU": iou,
            "PixelAccuracy": acc
        })

    if i < 3:
        img = sample_image[0].cpu().permute(1, 2, 0).numpy()
        gt = sample_target[0].cpu().numpy()

        fig, axs = plt.subplots(2, 4, figsize=(18, 9))
        axs[0, 0].imshow(img)
        axs[0, 0].set_title("Input Image")
        axs[0, 0].axis("off")

        axs[0, 1].imshow(apply_colormap(gt))
        axs[0, 1].set_title("GT")
        axs[0, 1].axis("off")

        axs[0, 2].imshow(apply_colormap(modes["Single"][0].cpu().numpy()))
        axs[0, 2].set_title("Single UNet")
        axs[0, 2].axis("off")

        axs[0, 3].imshow(apply_colormap(modes["Ensemble"][0].cpu().numpy()))
        axs[0, 3].set_title("Ensemble UNet")
        axs[0, 3].axis("off")

        axs[1, 0].imshow(apply_colormap(modes["FGSM_Single"][0].cpu().numpy()))
        axs[1, 0].set_title("FGSM + Single")
        axs[1, 0].axis("off")

        axs[1, 1].imshow(apply_colormap(modes["PGD_Single"][0].cpu().numpy()))
        axs[1, 1].set_title("PGD + Single")
        axs[1, 1].axis("off")

        axs[1, 2].imshow(apply_colormap(modes["FGSM_Ensemble"][0].cpu().numpy()))
        axs[1, 2].set_title("FGSM + Ensemble")
        axs[1, 2].axis("off")

        axs[1, 3].imshow(apply_colormap(modes["PGD_Ensemble"][0].cpu().numpy()))
        axs[1, 3].set_title("PGD + Ensemble")
        axs[1, 3].axis("off")

        plt.tight_layout()
        plt.savefig(f"sample_{i}_viz.png")
        plt.close()

df = pd.DataFrame(results)
df.to_csv("cityscapes_segmentation_metrics.csv", index=False)
print("Saved results to cityscapes_segmentation_metrics.csv")

/var/folders/b6/486jxbhd03z06w9634qqg4km0000gn/T/ipykernel_19407/2007709468.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  single_unet.load_state_dict(torch.load("./we

Saved results to cityscapes_segmentation_metrics.csv


In [ ]:
import pandas as pd

df = pd.read_csv("cityscapes_segmentation_metrics.csv")

summary_df = df.groupby("Mode")[["Dice", "IoU", "PixelAccuracy"]].mean().reset_index()
summary_df.to_csv("cityscapes_segmentation_metrics_summary.csv", index=False)

with open("cityscapes_segmentation_metrics.csv", "a") as f:
    f.write("\n# Summary (Average metrics per mode)\n")
    summary_df.to_csv(f, index=False)

print("Saved detailed metrics and summary to CSV files.")

Saved detailed metrics and summary to CSV files.


In [8]:
summary_df

,Mode,Dice,IoU,PixelAccuracy
0,Ensemble,0.200513,0.174177,0.872455
1,FGSM_Ensemble,0.137818,0.108745,0.655298
2,FGSM_Single,0.098644,0.071146,0.383289
3,PGD_Ensemble,0.138914,0.106981,0.635577
4,PGD_Single,0.049675,0.033256,0.177698
5,Single,0.194715,0.167594,0.852722
